In [ ]:
%load_ext autoreload
%autoreload 2
from awesome.run.awesome_config import AwesomeConfig
from awesome.run.awesome_runner import AwesomeRunner
from awesome.util.reflection import class_name
from awesome.analytics.result_model import ResultModel
from awesome.util.path_tools import get_project_root_path, get_package_root_path
import os
import torch
import re
from awesome.util.format import latex_postprocessor
import pandas as pd
from typing import Any, Dict
os.chdir(get_project_root_path()) # Beeing in the root directory of the project is important for the relative paths to work consistently

In [ ]:
from awesome.analytics.result_comparison import ResultComparison

paths = [
         "./runs/fbms_local/eval/unet/joint_realnvp/2024-01-11",
         "./runs/fbms_local/eval/unet/joint_realnvp/2024-01-11-seed"
         ]
models = []

for path in paths:
    for folder in os.listdir(path):
        if folder.startswith("old") or folder.startswith("log"):
            continue
        model = ResultModel.from_path(os.path.join(path, folder))
        models.append(model)

import re
p = r"#?(?P<cfg_num>\d+)?_?(?P<net>[A-z0-9]+)_?(?P<feat>(\+\w+)*)\_(?P<date>\d{2}_\d{2}_\d{2})\_(?P<time>\d{2}_\d{2}_\d{2})"
pattern = re.compile(p)

for model in models:
    match = pattern.match(model.name)
    model_name = None
    feat = []
    if match:
        model_name = match.group('net').strip("_")
        features = match.group('feat')
        if features is not None and features != "":
            feat = features.strip("+").split("+")
            if not any(["seed" in x for x in feat]):
                feat.append("seed42")
    else:
        print('No match for', model.name)
    model_name = model_name.replace("NET", "Net")
    model.display_name = model_name + " " + " ".join(feat)
    model.features = list(feat)
    model.config.result_directory = "final_mask"
    model.save_config()


# Resort the models by name to get a meaningful table order

_order = []

models = sorted(models, key=lambda m: _order.index(m.name) if m.name in _order else 0)

comparison = ResultComparison(models)
comparison.assign_numbers(force=True)

os.environ['PLOT_OUTPUT_DIR'] = comparison.output_folder

save_args = dict(transparent=False, save=True, dpi=300, ext=["png", "pdf"])

models

In [ ]:
from typing import Tuple

metrics = [
    "eval/epoch/MeanForegroundBinaryMIOU" ,
    "eval/epoch/MeanPriorForegroundBinaryMIOU",
    "eval/epoch/MeanPixelAccuracy",
    "eval/epoch/MeanPriorPixelAccuracy",
    "eval/epoch/MeanCRFForegroundBinaryMIOU",
    "eval/epoch/MeanCRFPixelAccuracy",
]

col_mapping = {
    "eval/epoch/MeanForegroundBinaryMIOU": "IoU",
    "eval/epoch/MeanCRFForegroundBinaryMIOU": "CRF IoU",
    "eval/epoch/MeanPixelAccuracy": "Acc.",
    "eval/epoch/MeanCRFPixelAccuracy": "CRF Acc.",
    "eval/epoch/MeanPriorPixelAccuracy" : "Prior Acc.",
    "eval/epoch/MeanPriorForegroundBinaryMIOU": "Prior IoU" 
}

index_mapping = {
    0: "Baseline",
    15: "Joint"
}

new_colmapping = {}
for k, v in dict(col_mapping).items():
    for k_i, v_i in dict(index_mapping).items():
        new_colmapping[k + "_" + str(k_i)] = v + " " + v_i

col_mapping = new_colmapping

def extract_features(model: ResultModel) -> Dict[str, Any]:
    res = dict()
    
    res['joint'] = "joint" in model.features
    
    res['model_name'] = model.config.name.split(" ")[0]
    model_features = list(model.features)
    
    if res['joint']:
        model_features.remove("joint")
    
    seed = next((x for x in model_features if "seed" in x), None)
    if seed is None:
        seed = model.run_config.seed
        res['seed'] = seed
    else:
        model_features.remove(seed)
        res['seed'] = int(seed.replace("seed", ""))

    if "REFIT" in model_features:
        model_features.remove("REFIT")
        res['prior'] = "refit"

    if "original" in model_features:
        model_features.remove("original")
        res['prior'] = "original"
    if "retrain" in model_features:
        model_features.remove("retrain")
        res['prior'] = "refit"
    if "retrain_xy" in model_features:
        model_features.remove("retrain_xy")
        res['prior'] = "refit"

    elif "convex" in model_features:
        model_features.remove("convex")
        res['prior'] = "convex"
    elif "diffeo" in model_features:
        model_features.remove("diffeo")
        res['prior'] = "diffeo"
    else:
        res['prior'] = "none"

    if "only_prior" in model_features:
        model_features.remove("only_prior")
        res['prior'] = res['prior'] + "+only_prior"

    if "all_frames" in model_features:
        model_features.remove("all_frames")
        res['prior'] = res['prior'] + "+all_frames"
    if "deeper" in model_features:
        model_features.remove("deeper")
        res['prior'] = res['prior'] + "+deeper"

    dataset_name = model_features.pop(0)
    res['dataset_name'] = dataset_name

    assert len(model_features) == 1, f"Multiple features {model_features} in model {model.output_path}"
    res['feature_type'] = model_features[0]

    return res

df = comparison.metric_table(metrics, 
                             ref="all", 
                             mode="max",
                        formatting=False)

df = df.reset_index()

def extract_ft_row(row: pd.Series) -> Tuple[str, bool, str, int]:
    name = row['index']
    model = [m for m in models if m.name == name][0]
    res = extract_features(model)
    return (res['model_name'], res['joint'], res['feature_type'], res['seed'], res['dataset_name'], res['prior'])

df[['model_name', 'joint', 'feature_type', 'seed', 'dataset_name', "prior"]] = df.apply(extract_ft_row, axis=1, result_type="expand")

df = df[['model_name', 'dataset_name'] + list(col_mapping.keys()) + ['joint', 'feature_type', 'seed', "prior"]]

grouped = df.groupby(['model_name', 'joint', 'feature_type'])

display_df = df.rename(columns=col_mapping)
display(display_df)

In [ ]:
display_df.describe()

# Seed grouping

In [ ]:
group_by = ['model_name', 'dataset_name', 'joint', 'feature_type', "prior"]

grouped = display_df.groupby(group_by)
display_df = grouped.mean().reset_index()
display(display_df)

In [ ]:
display_df

# Summarized Results of Path Connectedness

In [ ]:
display_columns = ["IoU Baseline", "IoU Joint", "Prior IoU Baseline", "Prior IoU Joint", "Acc. Baseline", "Acc. Joint", "Prior Acc. Baseline", "Prior Acc. Joint"]

numbers = display_df[display_columns].describe().loc["mean"].to_frame().T

def subst(x):
    x = re.sub("(?P<name>.*) Baseline", "Baseline \g<1>", x)
    x = re.sub("(?P<name>.*) Joint", "Joint \g<1>", x)
    return x

def subs_stubs(x):
    return x.replace("Joint ", "").replace("Baseline ", "")
new_col = [subst(x) for x in display_columns]
# Group joint metrics

new_numbers = pd.DataFrame(index=numbers.index, columns=new_col)
new_numbers[new_col] = numbers[display_columns]

new_numbers = new_numbers.rename(index=dict(mean="Sequential"))

joints_cols = [x for x in new_numbers.columns if "Joint" in x]
base_line_columns = [x for x in new_numbers.columns if "Baseline" in x]

joint_metrics = {subs_stubs(k): v for k, v in new_numbers[joints_cols].iloc[0].to_dict().items()}
sequential_metrics = {subs_stubs(k): v for k, v in  new_numbers[base_line_columns].iloc[0].to_dict().items()}

order = ["IoU", "Prior IoU", "Acc.", "Prior Acc."]

metric_df = pd.DataFrame(columns=order, index=["Sequential", "Joint"])
metric_df.loc["Sequential"] = [sequential_metrics[x] for x in order]
metric_df.loc["Joint"] = [joint_metrics[x] for x in order]


wide_numbers = pd.wide_to_long(new_numbers.reset_index(), stubnames=["Joint", "Baseline"], i="index", j="model", sep=" ").reset_index()

display(metric_df)

# Extended version for appendix

In [ ]:
extended_columns = ["dataset_name", "IoU Baseline", "Prior IoU Baseline", "Acc. Baseline", "Prior Acc. Baseline", "IoU Joint", "Prior IoU Joint", "Acc. Joint", "Prior Acc. Joint"]

def fnc(label):
    approach = None
    network = None
    s = label
    if "Baseline" in label:
        approach = "Sequential"
        s = s.replace("Baseline", "")
    else:
        approach = "Joint"
        s = s.replace("Joint", "")
    if "Prior" in label:
        network = "Prior"
        s = s.replace("Prior", "")
    else:
        network = "UNet"

    return approach, s.strip(), network

tuple_cols_mapping = {x: fnc(x) for x in extended_columns[1:]}
tuple_cols_mapping_inverted = {v: k for k, v in tuple_cols_mapping.items()}

extended_df = display_df[extended_columns]

data_cols = pd.MultiIndex.from_tuples(tuple_cols_mapping.values(), names=["Approach", "Metric", "Network"])

index_order = list(data_cols)
index_order_old = ["dataset_name"] + [tuple_cols_mapping_inverted.get(k, k) for k in index_order]

df_joint_mi_cols = extended_df[index_order_old].copy()
df_joint_mi_cols.set_index("dataset_name", inplace=True)

df_joint_mi_cols.index.name = "Sequence"

df_joint_mi_cols.sort_index(inplace=True)

df_joint_mi_cols.columns = data_cols
df_joint_mi_cols = df_joint_mi_cols.round(decimals=3)
df_joint_mi_cols


In [ ]:
print(df_joint_mi_cols.to_latex(float_format="{:0.3f}".format))